In [10]:
import pandas as pd
import matplotlib.pyplot as mltp
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import sklearn as sk
import seaborn as sea
import re 
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

# LDA

In [11]:
import numpy as np
import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import pickle 

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)
from typing import Dict, List
import spacy
from string import punctuation
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline

In [12]:
stopwords = stopwords.words('english')

In [13]:
import yaml
def load_markdown_file(file_path):
    with open(file_path, "r") as stream:
        markdown_str = stream.read()
        return markdown_str

def load_yaml_file(file_path):
    # reads the yml files as a dictionary, were each topic is a key and the values are a list of elements
    with open(file_path, "r") as stream:
        yaml_dict = yaml.safe_load(stream)
        return yaml_dict



def _add_sentence_to_list(sentence: str, sentences_list):
    """
    Add a sentence to the list of sentences.
    Args:
        sentence (str):
            Sentence to be added.
        sentences (List[str]):
            List of sentences.
    """
    while sentence.startswith(" "):
        # remove leading space
        sentence = sentence[1:]
    if all(c in punctuation for c in sentence) or len(sentence) == 1:
        # skip sentences with only punctuation
        return
    sentences_list.append(sentence)


def get_sentences(text: str):
    """
    Get sentences from a text.
    Args:
        text (str):
            Text to be processed.
    Returns:
        List[str]:
            List of sentences.
    """
    # get the paragraphs
    text=   re.sub(" \d+\n", ".", text)
    text=   re.sub("\n\d+", " ", text)
    text=   re.sub("\n", " ", text)
    text=   re.sub("\d+.", "", text)
    paragraphs = re.split(r' *[\.\?!][\'"\)\]]* *', text)
    paragraphs = [p for p in paragraphs if p != ""]
    # get the sentences from the paragraphs
    sentences = list()
    for paragraph in paragraphs:
        if paragraph.startswith("#"):
            _add_sentence_to_list(paragraph, sentences)
            continue
        prev_sentence_idx = 0
        for idx in range(len(paragraph)):
            if idx + 1 < len(paragraph):
                if (paragraph[idx] == "." and not paragraph[idx + 1].isdigit()) or (
                    paragraph[idx] in "!?"
                ):
                    sentence = paragraph[prev_sentence_idx : idx + 1]
                    _add_sentence_to_list(sentence, sentences)
                    prev_sentence_idx = idx + 1
            else:
                sentence = paragraph[prev_sentence_idx:]
                _add_sentence_to_list(sentence, sentences)
    return sentences

In [14]:
DATA_DIR='data/raw'

In [15]:
import os
data_names = os.listdir(DATA_DIR)
data_names = [name[:-4] for name in data_names if name != ".DS_Store"]
data_names = list(filter(None, data_names))

In [16]:
data_names

['DIE_GRUENEN_Wahlprogramm_2021',
 'FDP_Wahlprogramm_2021',
 'DIE_LINKE_Wahlprogramm_2021',
 'CDU-CSU_Wahlrprogramm_2021',
 'SPD_Wahlprogramm_2021',
 'AFD_Wahlprogramm_2021']

In [17]:
text_liste=[]
dic_list={}
for element in tqdm(list(filter(None, data_names))):
    program_txt = load_markdown_file(f"data/raw/{element}.txt")
    sentences = get_sentences(program_txt)
    for item in sentences:
        text_liste.append(item)
    dic_list[element]=sentences

  0%|          | 0/6 [00:00<?, ?it/s]

In [18]:

text_liste[0][0:90]

'IEITTESALTENTR Alles ist drin'

SpaCy ist eine beliebte Open-Source-Bibliothek für die Verarbeitung natürlicher Sprache (NLP) in Python. Sie enthält unter anderem Werkzeuge für Tokenisierung, Part-of-Speech-Tagging, Dependency Parsing und Named Entity Recognition. Diese Werkzeuge sind in einer Pipeline organisiert, die den Eingabetext verarbeitet und eine Vielzahl von linguistischen Merkmalen extrahiert, die für eine Vielzahl von NLP-Aufgaben verwendet werden können.

Der erste Schritt in der SpaCy-Pipeline ist die Tokenisierung, d. h. die Zerlegung des Eingabetextes in einzelne Wörter (Token). Dies geschieht anhand einer Reihe von Regeln, die Interpunktion, Kontraktionen und andere Sonderfälle berücksichtigen. Die so entstandenen Token werden dann mit verschiedenen linguistischen Merkmalen versehen, z. B. mit ihrem Part-of-Speech-Tag und ihrem Lemma (Grundform).

Der nächste Schritt in der Pipeline ist das Part-of-Speech-Tagging, bei dem jedes Token mit seinem Part of Speech (z. B. Substantiv, Verb oder Adjektiv) gekennzeichnet wird. Dies geschieht in der Regel mithilfe eines maschinellen Lernmodells, das auf einem großen annotierten Textkorpus trainiert wurde. Das Part-of-Speech-Tagging ist ein wichtiger Schritt, da es dem Modell ermöglicht, die grammatikalische Rolle jedes Wortes im Satz zu identifizieren, was für Aufgaben wie Dependency Parsing und Named Entity Recognition nützlich sein kann.

Beim Dependency Parsing werden die Beziehungen zwischen Wörtern in einem Satz identifiziert, z. B. Subjekt-Verben-Beziehungen und Modifikator-Kopf-Beziehungen. Dies geschieht in der Regel mithilfe eines maschinellen Lernmodells, das auf einem großen annotierten Textkorpus trainiert wurde. Das Parsen von Abhängigkeiten ist ein wichtiger Schritt, da es dem Modell ermöglicht, die Struktur des Satzes und die Beziehungen zwischen den Wörtern zu verstehen, was für Aufgaben wie die Informationsextraktion und die maschinelle Übersetzung nützlich sein kann. Jedoch wird dies Aufgrund der Anwendung ncith verwendet, da dies meist mit den höchsten Aufwand erzeugt

Bei der Erkennung benannter Entitäten handelt es sich um den Prozess der Identifizierung und Klassifizierung benannter Entitäten in Texten, z. B. Personen, Organisationen und Orte. Dies geschieht in der Regel mithilfe eines maschinellen Lernmodells, das auf einem großen kommentierten Textkorpus trainiert wurde. Die Erkennung von benannten Entitäten ist ein wichtiger Schritt, da sie es dem Modell ermöglicht, strukturierte Informationen aus unstrukturiertem Text zu extrahieren, was für Aufgaben wie die Informationsextraktion und die Textklassifizierung nützlich sein kann.

Insgesamt ist die SpaCy-Pipeline ein leistungsfähiges Werkzeug zur Extraktion einer Vielzahl von linguistischen Merkmalen aus Text, die für eine Vielzahl von NLP-Aufgaben verwendet werden können. Durch die Kombination dieser Merkmale mit Modellen des maschinellen Lernens ist es möglich, anspruchsvolle NLP-Anwendungen zu erstellen, die natürlichsprachliche Texte analysieren und verstehen können.


In [19]:
def lemmatization_test(texts,  allowed_posttags=['NOUN','ADJ','VERB','ADV']):
    nlp=spacy.load('en_core_web_md',disable=['parser','ner'])
    texts_out=[]
    for text in tqdm(texts):
        doc= nlp(text)
        new_text=[]
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final=' '.join(new_text)
        texts_out.append(final)
    return texts_out

In [20]:
lemma_text=lemmatization_test(text_liste)

  0%|          | 0/18858 [00:00<?, ?it/s]

In [21]:
print(lemma_text[0][0:90])

ist drin


In [22]:
def gen_words(texts):
    final=[]
    for text in tqdm(texts):
        new= gensim.utils.simple_preprocess(text,deacc=True)
        final.append(new)
    return final

In [23]:
data_words=gen_words(lemma_text)

  0%|          | 0/18858 [00:00<?, ?it/s]

In [24]:
print(data_words[0][0:90])

['ist', 'drin']


### Bigrams and Trigrams

Ein n-Gramm ist eine zusammenhängende Sequenz von n Elementen aus einer bestimmten Text- oder Sprachprobe. N-Gramme sind in der natürlichen Sprachverarbeitung und Computerlinguistik weit verbreitet. Sie können für eine Vielzahl von Aufgaben verwendet werden, z. B. für die Sprachmodellierung, die Informationsbeschaffung und die maschinelle Übersetzung.

Bigramme sind Zwei-Wort-Folgen von Wörtern, die nacheinander in einem Text vorkommen. In dem Satz "die Katze im Hut" wären die Bigramme zum Beispiel "die Katze", "Katze in" und "in der". Bigramme können für Aufgaben wie Rechtschreibkorrektur, Sprachmodellierung und Informationsabfrage nützlich sein.

Trigramme sind Drei-Wort-Folgen von Wörtern, die nacheinander in einem Text vorkommen. In dem Satz "die Katze im Hut" wären die Trigramme zum Beispiel "die Katze in", "die Katze im" und "im Hut". Trigramme können für Aufgaben wie Sprachmodellierung und Informationsabfrage nützlich sein.

N-Gramme werden häufig in der natürlichen Sprachverarbeitung verwendet, da sie den Kontext und die Bedeutung von Wörtern in einem Text erfassen können. Beispielsweise ist das Bigramm "New York" informativer als die einzelnen Wörter "New" und "York", da es darauf hindeutet, dass die beiden Wörter miteinander verwandt sind und sich wahrscheinlich auf die Stadt beziehen. In ähnlicher Weise ist das Trigramm "die Katze in" informativer als die Bigramme "die Katze" und "Katze in", da es darauf hindeutet, dass die Wörter Teil einer Sequenz sind und wahrscheinlich in irgendeiner Weise miteinander in Beziehung stehen.

Insgesamt können n-Gramme ein nützliches Werkzeug für das Verständnis und die Verarbeitung von Sprachdaten sein, und Bigramme und Trigramme werden häufig in einer Vielzahl von Aufgaben der natürlichen Sprachverarbeitung verwendet.


In [25]:
bigram_phrases=gensim.models.Phrases(data_words,min_count=3,threshold=100)
trigram_phases=gensim.models.Phrases(bigram_phrases[data_words],threshold=50)

bigram=gensim.models.phrases.Phraser(bigram_phrases)
trigram=gensim.models.phrases.Phraser(trigram_phases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trgram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams=make_bigrams(data_words)
data_bigrams_trigrams=make_trgram(data_bigrams)

In [26]:
from gensim.models import TfidfModel

id2word=corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus=[id2word.doc2bow(text) for text in texts]

print(corpus[0][0:90])

tfidf=TfidfModel(corpus,id2word=id2word)

low_value=0.03
words=[]
words_missing_in_tfdf=[]

for i in tqdm(range(0,len(corpus))):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops=low_value_words+words_missing_in_tfdf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i]=new_bow


[(0, 1), (1, 1)]


  0%|          | 0/18858 [00:00<?, ?it/s]

Ein wichtiger aber auch schwieriger Aspekt des LDA prozesses ist die Bestimmung der ANzhal der Topics. Hierbei ist wie bei Maschine Learning eine autonmatische Verbesserung und Lernen nicht gegeben, weshlab durch erforschen und Analysen die Optimalen Parameter gefunden werden müssen. Dies ist besonders schweirig, da meist Klassen entstehen, welche eine hohe Anzahl an Elementen enthalten

In [27]:
topics = load_yaml_file('topic_modeling/topic_g.yml')
list(topics.keys())

['wirtschaft',
 'Klima',
 'Bildung',
 'Gesundheit',
 'Wissenschaft',
 'soziale Ursachen',
 'Politik und Ideologie',
 'Infrastruktur']

In [28]:
num_topics=len(list(topics.keys()))

In [29]:
lda_model=gensim.models.ldamodel.LdaModel(
corpus=corpus,
id2word=id2word,
num_topics=num_topics,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto'
)

In [30]:
dic={}
dic_sum={}
for idx, topic in lda_model.print_topics(num_topics,num_words=100):
    elements=[]
    percent=[]

    for e in topic.split('+'):
        elements.append(e.split('*')[1].replace('"','').strip())
        percent.append(float(e.split('*')[0].replace('"','').strip()))

    dic[str(idx)+'_word']=elements
    dic[str(idx)+'_per']=percent
    dic_sum[idx]=sum(percent)
print(dic_sum)

{0: 0.7370000000000003, 1: 0.7860000000000003, 2: 0.7230000000000004, 3: 0.6790000000000003, 4: 0.7360000000000003, 5: 0.8840000000000003, 6: 0.7330000000000004, 7: 0.6930000000000004}


Zur bestimmung der optimalen Anzahl von relevanten Wörtern wurden daher auf 100 gestzt, um nah an 100% zu sein. Da je nach Wort eine untershciedliche Wahrscheinlichkeit und Somit gewichtung für das eine oder andere Topic entsteht

Daher werden die Wörter im nahchinein genutzt, um eine zurodnung durch abgleich und multiplikation der Gewichtung zu erziehlen, um basierend davon jenes Topic zu deklarieren, welches am Wahrscheinlichsten ist. Aus diesem Grund dies aus dem Wert der Summe der auftretenden Werte verglichen mit der Summe der Unterschiedlichen Gewichte mäglicher alternastiven Topics
Dafür wird volgendes getan:
1. Identifizieren, welche Wörter eines Topics und wei oft in einem Text enthalten sind
2. Basierend auf diesen die Wahrscheinlihckeiten bestimmen
3. Diese Addieren und den höchsten Wert identifizieren, welche das Wahrschienlichste Topic entspricht


In [31]:
df_topic=pd.DataFrame(data=dic)
df_topic.head()

,0_word,0_per,1_word,1_per,2_word,2_per,3_word,3_per,4_word,4_per,5_word,5_per,6_word,6_per,7_word,7_per
0,wollen,0.075,ist,0.184,auch,0.122,starken,0.037,werden,0.147,die,0.215,fur,0.278,dem,0.110
1,oder,0.069,nicht,0.112,muss,0.080,ihrer,0.034,mussen,0.081,und,0.150,es,0.028,ihre,0.053
2,wird,0.067,nur,0.061,al,0.068,unter,0.026,haben,0.050,der,0.132,unseren,0.020,vor,0.044
3,wir,0.061,sowie,0.043,un,0.030,ohne,0.026,sollen,0.040,den,0.042,uber,0.019,unterstutzen,0.035
4,fordern,0.051,zur,0.033,unserer,0.028,deutsche,0.025,soll,0.032,zu,0.039,dazu,0.017,noch,0.028


In [32]:
def create_df_overview(data_words,num_topics,df_topic):
    topic_of_text=[]
    for text in tqdm(data_words):
        liste_text_top=[]
        for t in range(num_topics):
            liste_top=[]
            for element in df_topic[f'{t}_word'].tolist():
                liste_top.append(len([i for i, e in enumerate(text) if e == element]))
    
            Result = []
            for i1, i2 in zip(liste_top, df_topic[f'{t}_per'].tolist()):
                Result.append(i1*i2)
            liste_text_top.append(sum(Result))    
        if sum(liste_text_top)!=0:
            topic_of_text.append([max(range(len(liste_text_top)), key=liste_text_top.__getitem__),max(liste_text_top)/sum(liste_text_top),liste_text_top])
        else:
            topic_of_text.append(['None','None',liste_text_top])
    return pd.DataFrame(data=topic_of_text,columns=['topic_num','percent','perc_list'])
       


In [33]:
pd.DataFrame(data=dic).to_csv(f'LDA/lda_gesamt_{num_topics}.csv')

In [34]:
# overview=pd.merge(df[['text','topic','gender','age']], create_df_overview(data_words,num_topics,df_topic), left_index=True, right_index=True)
# overview.head()

NameError: name 'df' is not defined

- ml --> prediction welches richtige topic man hat --> meist mehr topics --> ggf noch pca --> (downstream task) --> klassifikation
- einrichtigen text --> grundlagen teil und praxis teil 

In [ ]:
# Build App
import socket
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output

In [ ]:

def vis_topic(overview):
    sock = socket.socket()
    sock.bind(('', 0))
    sock.getsockname()[1]
    app = JupyterDash(__name__)
    app.layout = html.Div([
        html.H1("Übersicht der Verteilungen basierend auf den initialen Topics"),
        dcc.Graph(id='graph'),
        html.Label([
            "Topics",
            dcc.Dropdown(
                id='colorscale-dropdown', clearable=False,
                value='plasma', options=[
                    {'label': c, 'value': c}
                    for c in overview['topic'].unique().tolist()
                ]
                )
        ]),
    ])
    # Define callback to update graph
    @app.callback(
        Output('graph', 'figure'),
        [Input("colorscale-dropdown", "value")]
    )
    def update_figure(colorscale):
        testdf = overview[overview['topic']==colorscale].topic_num.value_counts().rename_axis('unique_values').reset_index(name='counts')
        return px.pie(testdf, values='counts', names='unique_values')
    # Run app and display result inline in the notebook
    app.run_server(mode='inline', port=sock.getsockname()[1])


In [ ]:
def vis_topic_num(overview):
    sock = socket.socket()
    sock.bind(('', 0))
    sock.getsockname()[1]
    app = JupyterDash(__name__)
    app.layout = html.Div([
        html.H1("Übersicht der Verteilungen basierend auf den initialen Topics"),
        dcc.Graph(id='graph'),
        html.Label([
            "Topics",
            dcc.Dropdown(
                id='colorscale-dropdown', clearable=False,
                value='plasma', options=[
                    {'label': c, 'value': c}
                    for c in overview['topic_num'].unique().tolist()
                ]
                )
        ]),
    ])
    # Define callback to update graph
    @app.callback(
        Output('graph', 'figure'),
        [Input("colorscale-dropdown", "value")]
    )
    def update_figure(colorscale):
        testdf = overview[overview['topic_num']==colorscale].topic.value_counts().rename_axis('topic').reset_index(name='counts')
        return px.pie(testdf, values='counts', names='topic')
    # Run app and display result inline in the notebook
    app.run_server(mode='inline', port=sock.getsockname()[1])

In [ ]:
#vis_topic(overview)

In [ ]:
#vis_topic_num(overview)

In [ ]:
for idx, topic in lda_model.print_topics(num_topics,num_words=30):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.121*"sleep" + 0.096*"water" + 0.076*"drink" + 0.060*"type" + 0.058*"wake" + 0.045*"kinda" + 0.035*"pm" + 0.034*"am" + 0.022*"pizza" + 0.022*"evil" + 0.021*"milk" + 0.020*"chair" + 0.020*"board" + 0.019*"wash" + 0.017*"dish" + 0.013*"nasty" + 0.012*"perform" + 0.011*"repeat" + 0.011*"randomly" + 0.010*"pig" + 0.010*"cream" + 0.010*"firework" + 0.009*"btw" + 0.009*"watch_tv" + 0.009*"ppl" + 0.008*"lip" + 0.007*"tonite" + 0.007*"hall" + 0.007*"plenty" + 0.007*"afterwards"
Topic: 1 
Words: 0.119*"part" + 0.066*"learn" + 0.049*"experience" + 0.046*"group" + 0.046*"student" + 0.044*"busy" + 0.037*"apparently" + 0.036*"lead" + 0.031*"personal" + 0.030*"member" + 0.030*"touch" + 0.026*"build" + 0.019*"treat" + 0.017*"lack" + 0.017*"certainly" + 0.015*"got" + 0.014*"horrible" + 0.013*"rise" + 0.013*"station" + 0.013*"language" + 0.012*"pocket" + 0.012*"freedom" + 0.012*"button" + 0.011*"ignore" + 0.011*"concern" + 0.010*"education" + 0.008*"holy" + 0.008*"proper" + 0.008*"act

## Visualisation

In [ ]:

import pyLDAvis.gensim_models as gensimvis
import pickle 
pyLDAvis.enable_notebook()
import os
LDAvis_data_filepath = os.path.join('LDA/pic/ges_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word,mds='mmds',R=30)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, 'LDA/pic/ges_'+ str(num_topics) +'.html')
LDAvis_prepared

/Users/niclascramer/opt/miniconda3/envs/NLP/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/niclascramer/opt/miniconda3/envs/NLP/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.462190  0.106758       1        1  46.887151
2      0.251755  0.301896       2        1  10.015108
0      0.367632 -0.095827       3        1   8.537644
3      0.148419  0.018304       4        1   7.701462
1     -0.212261 -0.303703       5        1   7.506453
4     -0.087653  0.328166       6        1   7.243174
6      0.115623 -0.340898       7        1   7.083253
7     -0.121324 -0.014696       8        1   5.025755, topic_info=             Term          Freq         Total Category  logprob  loglift
4             die  11516.000000  11516.000000  Default  30.0000  30.0000
43            und   8030.000000   8030.000000  Default  29.0000  29.0000
73            fur   2246.000000   2246.000000  Default  28.0000  28.0000
86            der   7082.000000   7082.000000  Default  27.0000  27.0000
1             ist   1577.000000   1577.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
1183  stattdessen     37.806233     38.701504   Topic8  -5.0213   2.9672
608         hohen     34.622648     35.521832   Topic8  -5.1092   2.9650
2018     wichtige     34.026424     34.920677   Topic8  -5.1266   2.9647
927         junge     33.968120     34.862510   Topic8  -5.1283   2.9646
922    verstarken     32.451674     33.347671   Topic8  -5.1740   2.9634

[274 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
1021       2  0.997125           ab
99         2  0.992671         aber
8285       1  0.999394          afd
10548      1  0.449644  afd_fordert
10548      2  0.080898  afd_fordert
...      ...       ...          ...
188        1  0.999782           zu
241        3  0.991966          zum
286        3  0.992936    zunehmend
465        5  0.998549          zur
57         2  0.986616     zwischen

[249 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 1, 4, 2, 5, 7, 8])

# Nur Nomen und Verben

In [ ]:
lemma_text_NV=lemmatization_test(text_liste,['NOUN','VERB'])

  0%|          | 0/18858 [00:00<?, ?it/s]

In [ ]:
data_words_NV=gen_words(lemma_text_NV)

  0%|          | 0/18858 [00:00<?, ?it/s]

In [ ]:
bigram_phrases_NV=gensim.models.Phrases(data_words_NV,min_count=3,threshold=100)
trigram_phases_NV=gensim.models.Phrases(bigram_phrases_NV[data_words_NV],threshold=50)

bigram_NV=gensim.models.phrases.Phraser(bigram_phrases_NV)
trigram_NV=gensim.models.phrases.Phraser(trigram_phases_NV)

def make_bigrams(bigram,texts):
    return [bigram[doc] for doc in texts]

def make_trgram(trigram,bigram,texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams_NV=make_bigrams(bigram_NV,data_words_NV)
data_bigrams_trigrams_NV=make_trgram(trigram_NV,bigram_NV,data_bigrams_NV)

In [ ]:
from gensim.models import TfidfModel

id2word=corpora.Dictionary(data_bigrams_trigrams_NV)

texts = data_bigrams_trigrams_NV

corpus=[id2word.doc2bow(text) for text in texts]

print(corpus[0][0:90])

tfidf=TfidfModel(corpus,id2word=id2word)

low_value=0.03
words=[]
words_missing_in_tfdf=[]

for i in tqdm(range(0,len(corpus))):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops=low_value_words+words_missing_in_tfdf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i]=new_bow


[(0, 1)]


  0%|          | 0/18858 [00:00<?, ?it/s]

In [ ]:
lda_model=gensim.models.ldamodel.LdaModel(
corpus=corpus,
id2word=id2word,
num_topics=num_topics,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto'
)

In [ ]:
dic={}
dic_sum={}
for idx, topic in lda_model.print_topics(num_topics,num_words=100):
    elements=[]
    percent=[]

    for e in topic.split('+'):
        elements.append(e.split('*')[1].replace('"','').strip())
        percent.append(float(e.split('*')[0].replace('"','').strip()))

    dic[str(idx)+'_word']=elements
    dic[str(idx)+'_per']=percent
    dic_sum[idx]=sum(percent)
print(dic_sum)

{0: 0.7440000000000004, 1: 0.6870000000000003, 2: 0.7470000000000003, 3: 0.7640000000000003, 4: 0.8040000000000003, 5: 0.7340000000000003, 6: 0.8540000000000002, 7: 0.8500000000000002}


In [ ]:
pd.DataFrame(data=dic).to_csv(f'LDA/lda_nv_{num_topics}.csv')

In [ ]:
pd.DataFrame(data=dic)

,0_word,0_per,1_word,1_per,2_word,2_per,3_word,3_per,4_word,4_per,5_word,5_per,6_word,6_per,7_word,7_per
0,al,0.088,durch,0.221,fur,0.250,oder,0.078,zu,0.188,werden,0.199,der,0.351,die,0.332
1,mussen,0.077,sowie,0.059,wollen,0.063,auch,0.071,ist,0.119,dem,0.102,eine,0.114,und,0.185
2,wird,0.066,es,0.047,ihrer,0.038,unsere,0.064,nicht,0.106,darf,0.043,mit,0.081,den,0.066
3,mehr,0.032,gild,0.024,deutsche,0.028,hat,0.061,nur,0.050,unter,0.035,einer,0.055,afd,0.035
4,damit,0.030,mindesten,0.022,immer,0.025,unserer,0.045,haben,0.040,kapitel,0.031,bei,0.038,sind,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,sorgt,0.002,nichtstaatliche,0.000,festgelegten,0.001,einkomman,0.001,rund,0.001,familienpolitik,0.000,afd_bekennt,0.001,gestalten,0.001
96,nachhaltig,0.002,zustimman,0.000,konkreten,0.001,zuzulassen,0.001,fahren,0.001,eroffnet,0.000,weltweiten,0.001,bestimmte,0.001
97,gleich,0.002,heimischer,0.000,medizinischer,0.001,wirtschaftliche,0.001,weiterhin,0.001,grundlegenden,0.000,erweitern,0.001,abzulehnen,0.001
98,sichert,0.002,verbotene,0.000,familienpolitik,0.001,welcher,0.001,ideologisch,0.001,mobilitat,0.000,lokale,0.001,gibt,0.001


In [ ]:
overview_nomen= pd.merge(df[['text','topic','gender','age']], create_df_overview(data_words,num_topics,pd.DataFrame(data=dic)), left_index=True, right_index=True)
overview_nomen.head()

  0%|          | 0/19500 [00:00<?, ?it/s]

,text,topic,gender,age,topic_num,percent,perc_list
0,This blog is being posted due to the fact that...,Student,male,14,18,0.522646,"[0.0, 0.0, 0.0, 0.0, 0.012, 0.0, 0.0, 0.0, 0.0..."
1,So I have a big fucking interview tomorrow for...,Student,male,15,8,0.400248,"[0.0, 0.0, 0.0, 0.0, 0.055999999999999994, 0.1..."
2,I was reminded just now of the time Ashley and...,Student,female,17,8,0.318367,"[0.084, 0.396, 0.0, 0.0, 0.14900000000000002, ..."
3,I was checking up on my cousin Dylan and Fanni...,Student,female,23,18,0.316395,"[0.0, 0.0, 0.025, 0.0, 0.018, 0.071, 0.015, 0...."
4,for the NME interview click urlLink part 1 and...,Student,female,23,22,0.93568,"[0.0, 0.0, 0.0, 0.0, 0.06, 0.026, 0.0, 0.0, 0...."


In [ ]:
vis_topic(overview_nomen)

In [ ]:
vis_topic_num(overview_nomen)

In [ ]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(lda_model,corpus,id2word,mds='mmds',R=30)
vis

/Users/niclascramer/opt/miniconda3/envs/NLP/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.289821  0.334409       1        1  37.617130
6      0.129761  0.394234       2        1  18.597044
4      0.375152 -0.149635       3        1   9.493408
0     -0.337992 -0.031103       4        1   9.193421
2      0.256156  0.113598       5        1   8.643080
3     -0.020222 -0.033884       6        1   6.257162
5     -0.204795 -0.304741       7        1   5.805745
1      0.091762 -0.322878       8        1   4.393010, topic_info=             Term          Freq         Total Category  logprob  loglift
3             die  11791.000000  11791.000000  Default  30.0000  30.0000
73            der   6180.000000   6180.000000  Default  29.0000  29.0000
39            und   6562.000000   6562.000000  Default  28.0000  28.0000
62            fur   2044.000000   2044.000000  Default  27.0000  27.0000
165            zu   1686.000000   1686.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
511   profitieren     18.507449     19.365704   Topic8  -5.4134   3.0798
1180       haufig     18.250544     19.111760   Topic8  -5.4274   3.0790
1517        ihnen     17.989130     18.849324   Topic8  -5.4418   3.0784
3799    befordert     17.793058     18.659691   Topic8  -5.4528   3.0776
3878  eingehalten     18.764725     19.816567   Topic8  -5.3996   3.0706

[279 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1435      6  0.996368           ab
85        6  0.989181         aber
7101      1  0.999135          afd
9057      1  0.441078  afd_fordert
9057      2  0.179458  afd_fordert
...     ...       ...          ...
165       3  0.999229           zu
209       7  0.993213          zum
247       4  0.978453    zunehmend
2391      8  0.981154   zunehmende
387       1  0.997185          zur

[252 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 5, 1, 3, 4, 6, 2])

In [38]:
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trgram(texts):
    return [trigram[bigram[doc]] for doc in texts]
from gensim.models import TfidfModel

In [46]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
pyLDAvis.enable_notebook()
import os
def create_LDA(text_liste,num_topics,name,allowed_posttags=['NOUN','ADJ','VERB','ADV'],zusatz=''):
    lemma_text=lemmatization_test(text_liste,allowed_posttags)
    data_words=gen_words(lemma_text)
    bigram_phrases=gensim.models.Phrases(data_words,min_count=3,threshold=100)
    trigram_phases=gensim.models.Phrases(bigram_phrases[data_words],threshold=50)

    bigram=gensim.models.phrases.Phraser(bigram_phrases)
    trigram=gensim.models.phrases.Phraser(trigram_phases)

    data_bigrams=make_bigrams(data_words)
    data_bigrams_trigrams=make_trgram(data_bigrams)
  

    id2word=corpora.Dictionary(data_bigrams_trigrams)

    texts = data_bigrams_trigrams

    corpus=[id2word.doc2bow(text) for text in texts]

    print(corpus[0][0:90])

    tfidf=TfidfModel(corpus,id2word=id2word)

    low_value=0.03
    words=[]
    words_missing_in_tfdf=[]

    for i in tqdm(range(0,len(corpus))):
        bow = corpus[i]
        low_value_words = [] #reinitialize to be safe. You can skip this.
        tfidf_ids = [id for id, value in tfidf[bow]]
        bow_ids = [id for id, value in bow]
        low_value_words = [id for id, value in tfidf[bow] if value < low_value]
        drops=low_value_words+words_missing_in_tfdf
        for item in drops:
            words.append(id2word[item])
        words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

        new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
        corpus[i]=new_bow


    lda_model=gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto'
    )
    dic={}
    dic_sum={}
    for idx, topic in lda_model.print_topics(num_topics,num_words=100):
        elements=[]
        percent=[]

        for e in topic.split('+'):
            elements.append(e.split('*')[1].replace('"','').strip())
            percent.append(float(e.split('*')[0].replace('"','').strip()))

        dic[str(idx)+'_word']=elements
        dic[str(idx)+'_per']=percent
        dic_sum[idx]=sum(percent)
    df_topic=pd.DataFrame(data=dic)
    pd.DataFrame(data=dic).to_csv(f'LDA/{name}{zusatz}_{num_topics}.csv')
  
    LDAvis_data_filepath = os.path.join(f'LDA/pic/{name}'+zusatz+str(num_topics))
    # # this is a bit time consuming - make the if statement True
    # # if you want to execute visualization prep yourself
    if 1 == 1:
        LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word,mds='mmds',R=30)
        with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)
    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, f'LDA/pic/{name}'+zusatz+ str(num_topics) +'.html')
    LDAvis_prepared

In [47]:
list(dic_list.keys())

['DIE_GRUENEN_Wahlprogramm_2021',
 'FDP_Wahlprogramm_2021',
 'DIE_LINKE_Wahlprogramm_2021',
 'CDU-CSU_Wahlrprogramm_2021',
 'SPD_Wahlprogramm_2021',
 'AFD_Wahlprogramm_2021']

In [48]:
dic_list['DIE_GRUENEN_Wahlprogramm_2021'][0:10]

['IEITTESALTENTR Alles ist drin',
 'Bundestagswahlprogramm',
 'Bereit, weil Ihr es seid',
 'Dieses Bundestagswahlprogramm wurde auf der  Bundesdelegiertenkonferenz von BÜNDNIS DIE GRÜNEN beschlossen, die vom bis  Juni digital stattgefunden hat',
 'Herausgeberin:  BÜNDNIS DIE GRÜNEN Platz vor dem Neuen Tor',
 'Berlin  Telefon:  Fax: E-Mail: info@gruene',
 'de Internet: gruene',
 'de  V',
 ':  BÜNDNIS DIE GRÜNEN Annkathrin Schäfer  Platz vor dem Neuen Tor',
 'Berlin  Layout und Satz: Twentyfour Seven Creative Media Service GmbH, Berlin twentyfour-/']

In [49]:
for name in tqdm(list(dic_list.keys())):
    create_LDA(dic_list['DIE_GRUENEN_Wahlprogramm_2021'],num_topics,name,allowed_posttags=['NOUN','ADJ','VERB','ADV'])

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

[(0, 1), (1, 1)]


  0%|          | 0/4880 [00:00<?, ?it/s]

...

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

[(0, 1), (1, 1)]


  0%|          | 0/4880 [00:00<?, ?it/s]

/Users/niclascramer/opt/miniconda3/envs/NLP/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [50]:
for name in tqdm(list(dic_list.keys())):
    create_LDA(dic_list['DIE_GRUENEN_Wahlprogramm_2021'],num_topics,name,allowed_posttags=['NOUN','VERB'],zusatz='_verb_and_nome')

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

[(0, 1)]


  0%|          | 0/4880 [00:00<?, ?it/s]

...

  0%|          | 0/4880 [00:00<?, ?it/s]

  0%|          | 0/4880 [00:00<?, ?it/s]

[(0, 1)]


  0%|          | 0/4880 [00:00<?, ?it/s]

/Users/niclascramer/opt/miniconda3/envs/NLP/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
